# IMPORTANDO A BASE DE DADOS

In [2]:
import pandas as pd
import numpy as np

In [3]:
pd.set_option('display.width', 400)         # MOSTRAR TODAS AS COLUNAS HORIZONTALMENTE
pd.set_option('display.max_columns', None)  # NÃO HÁ LIMITE DE COLUNAS PRA MOSTRAR
pd.set_option('display.max_rows', 10)       # MOSTRA SEMPRE 10 LINHAS

In [4]:
df = pd.read_csv(r'CleanDataBase.csv')
df.head(5)

,Nome,Genero,MultiplasLinhas,ServicoInternet,ServicoSegurancaOnline,ServicoBackupOnline,TipoContrato,FormaPagamento,ProtecaoEquipamento,ServicoSuporteTecnico,ServicoStreamingTV,ServicoFilmes,Aposentado,Dependentes,ServicoTelefone,Casado,FaturaDigital,ValorMensal,TotalGasto,Mês de Assinatura,MesesComoCliente,Churn
0,Celio Nunes da Silva,Masculino,Sim,Fibra,Sim,Sim,2 Anos,BoletoEletronico,Sim,Sim,Sim,Sim,Não,Não,Não,Não,Não,122.00,122.00,2022-01-23,0,Não
1,Klauss Miranda Marchi,Masculino,Não,Fibra,Sim,Não,2 Anos,DebitoAutomatico,Não,Sem Internet,Sim,Não,Não,Não,Não,Não,Não,82.00,82.00,2022-01-23,0,Não
2,Isabella Braga,Masculino,Sim,Nao,SemInternet,SemInternet,2 anos,CartaoCredito,SemInternet,SemInternet,SemInternet,SemInternet,0,Sim,Sim,Sim,Nao,74.95,2869.85,2017-06-05,38,Sim
3,Gael Teixeira,Feminino,Nao,Fibra,Nao,Sim,Mensal,CartaoCredito,Sim,Nao,Sim,Nao,0,Nao,Sim,Nao,Sim,55.90,238.50,2021-01-25,4,Nao
4,Manuela Teixeira,Feminino,Sim,Fibra,Nao,Nao,Mensal,DebitoAutomatico,Nao,Nao,Nao,Nao,1,Nao,Sim,Sim,Sim,53.45,119.50,2018-10-21,2,Nao


# TRATAMENTO DOS VALORES DATA
A coluna "Mês de Assinatura" deve também ser convertida para o formato de data

In [5]:
df['Mês de Assinatura'] = pd.to_datetime(df['Mês de Assinatura'], errors='coerce')
df.head(6)

,Nome,Genero,MultiplasLinhas,ServicoInternet,ServicoSegurancaOnline,ServicoBackupOnline,TipoContrato,FormaPagamento,ProtecaoEquipamento,ServicoSuporteTecnico,ServicoStreamingTV,ServicoFilmes,Aposentado,Dependentes,ServicoTelefone,Casado,FaturaDigital,ValorMensal,TotalGasto,Mês de Assinatura,MesesComoCliente,Churn
0,Celio Nunes da Silva,Masculino,Sim,Fibra,Sim,Sim,2 Anos,BoletoEletronico,Sim,Sim,Sim,Sim,Não,Não,Não,Não,Não,122.00,122.00,2022-01-23,0,Não
1,Klauss Miranda Marchi,Masculino,Não,Fibra,Sim,Não,2 Anos,DebitoAutomatico,Não,Sem Internet,Sim,Não,Não,Não,Não,Não,Não,82.00,82.00,2022-01-23,0,Não
2,Isabella Braga,Masculino,Sim,Nao,SemInternet,SemInternet,2 anos,CartaoCredito,SemInternet,SemInternet,SemInternet,SemInternet,0,Sim,Sim,Sim,Nao,74.95,2869.85,2017-06-05,38,Sim
3,Gael Teixeira,Feminino,Nao,Fibra,Nao,Sim,Mensal,CartaoCredito,Sim,Nao,Sim,Nao,0,Nao,Sim,Nao,Sim,55.90,238.50,2021-01-25,4,Nao
4,Manuela Teixeira,Feminino,Sim,Fibra,Nao,Nao,Mensal,DebitoAutomatico,Nao,Nao,Nao,Nao,1,Nao,Sim,Sim,Sim,53.45,119.50,2018-10-21,2,Nao
5,Isadora Macedo,Masculino,Nao,DSL,Nao,Nao,Mensal,BoletoEletronico,Nao,Nao,Nao,Sim,0,Nao,Sim,Nao,Sim,49.85,3370.20,2019-02-09,70,Nao


# SEPARANDO A DATA
Criando 3 novas colunas com o dia, mês e ano do mês de assinatura, para isso precisamos usar a coluna de data no formato de string e depois separá-la pelo caractere ' - '

In [6]:
textoMeses = df['Mês de Assinatura'].astype(str)
textoMeses.head()

0    2022-01-23
1    2022-01-23
2    2017-06-05
3    2021-01-25
4    2018-10-21
Name: Mês de Assinatura, dtype: object

In [7]:
df[['Ano', 'Mês', 'Dia']] = textoMeses.str.split('-', expand=True)
df[['Mês de Assinatura', 'Dia', 'Mês', 'Ano']].head()

,Mês de Assinatura,Dia,Mês,Ano
0,2022-01-23,23,01,2022
1,2022-01-23,23,01,2022
2,2017-06-05,05,06,2017
3,2021-01-25,25,01,2021
4,2018-10-21,21,10,2018


# VALIDANDO O CONTRATO
Adicionando na coluna pagamento o status do usuário, ou seja, se o pagamento está em dia com usando a data da última vez que ele renovou o plano em Mês de Assinatura

In [8]:
import datetime as dt

In [9]:
def validarContrato(dia, mes, ano, validade):
    try:
        dia, mes, ano = int(dia), int(mes), int(ano)
    except:
        return pd.NA
    
    hoje = dt.datetime.now()
    dataTabela = dt.datetime(ano, mes, dia)
    
    diasPassados = (hoje - dataTabela).days  # DISTANCIA ENTRE COMPRA E DIA ATUAL EM DIAS
    
    if validade == '2 anos' and diasPassados > 365 * 2:
        return 'Atrasado'
        
    elif validade == 'Anual' and diasPassados > 365:
        return 'Atrasado' 
        
    elif validade == 'Mensal' and diasPassados > 31:
        return 'Atrasado'
        
    else:
        return 'Em Dia'

In [10]:
df['Pagamento'] = df.apply(lambda x: validarContrato(x['Dia'], x['Mês'], x['Ano'], x['TipoContrato']), axis=1)
df.head(6)

,Nome,Genero,MultiplasLinhas,ServicoInternet,ServicoSegurancaOnline,ServicoBackupOnline,TipoContrato,FormaPagamento,ProtecaoEquipamento,ServicoSuporteTecnico,ServicoStreamingTV,ServicoFilmes,Aposentado,Dependentes,ServicoTelefone,Casado,FaturaDigital,ValorMensal,TotalGasto,Mês de Assinatura,MesesComoCliente,Churn,Ano,Mês,Dia,Pagamento
0,Celio Nunes da Silva,Masculino,Sim,Fibra,Sim,Sim,2 Anos,BoletoEletronico,Sim,Sim,Sim,Sim,Não,Não,Não,Não,Não,122.00,122.00,2022-01-23,0,Não,2022,01,23,Em Dia
1,Klauss Miranda Marchi,Masculino,Não,Fibra,Sim,Não,2 Anos,DebitoAutomatico,Não,Sem Internet,Sim,Não,Não,Não,Não,Não,Não,82.00,82.00,2022-01-23,0,Não,2022,01,23,Em Dia
2,Isabella Braga,Masculino,Sim,Nao,SemInternet,SemInternet,2 anos,CartaoCredito,SemInternet,SemInternet,SemInternet,SemInternet,0,Sim,Sim,Sim,Nao,74.95,2869.85,2017-06-05,38,Sim,2017,06,05,Atrasado
3,Gael Teixeira,Feminino,Nao,Fibra,Nao,Sim,Mensal,CartaoCredito,Sim,Nao,Sim,Nao,0,Nao,Sim,Nao,Sim,55.90,238.50,2021-01-25,4,Nao,2021,01,25,Atrasado
4,Manuela Teixeira,Feminino,Sim,Fibra,Nao,Nao,Mensal,DebitoAutomatico,Nao,Nao,Nao,Nao,1,Nao,Sim,Sim,Sim,53.45,119.50,2018-10-21,2,Nao,2018,10,21,Atrasado
5,Isadora Macedo,Masculino,Nao,DSL,Nao,Nao,Mensal,BoletoEletronico,Nao,Nao,Nao,Sim,0,Nao,Sim,Nao,Sim,49.85,3370.20,2019-02-09,70,Nao,2019,02,09,Atrasado


Deletando as colunas Dia e Ano deixando apenas a coluna mês convertida para o nome por extenso

In [11]:
df = df.drop(columns=['Dia', 'Ano'])
df['Mês'] = pd.to_numeric(df['Mês'], errors='coerce')
df = df.dropna(how='any', axis=0)

In [12]:
def converterMes(mes):
    nomeMeses = ('Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto',
                 'Setembro', 'Outubro', 'Novembro', 'Dezembro')
    
    return nomeMeses[int(mes)-1]

In [13]:
df['Mês'] = df['Mês'].apply(converterMes)
df.head(6)

,Nome,Genero,MultiplasLinhas,ServicoInternet,ServicoSegurancaOnline,ServicoBackupOnline,TipoContrato,FormaPagamento,ProtecaoEquipamento,ServicoSuporteTecnico,ServicoStreamingTV,ServicoFilmes,Aposentado,Dependentes,ServicoTelefone,Casado,FaturaDigital,ValorMensal,TotalGasto,Mês de Assinatura,MesesComoCliente,Churn,Mês,Pagamento
0,Celio Nunes da Silva,Masculino,Sim,Fibra,Sim,Sim,2 Anos,BoletoEletronico,Sim,Sim,Sim,Sim,Não,Não,Não,Não,Não,122.00,122.00,2022-01-23,0,Não,Janeiro,Em Dia
1,Klauss Miranda Marchi,Masculino,Não,Fibra,Sim,Não,2 Anos,DebitoAutomatico,Não,Sem Internet,Sim,Não,Não,Não,Não,Não,Não,82.00,82.00,2022-01-23,0,Não,Janeiro,Em Dia
2,Isabella Braga,Masculino,Sim,Nao,SemInternet,SemInternet,2 anos,CartaoCredito,SemInternet,SemInternet,SemInternet,SemInternet,0,Sim,Sim,Sim,Nao,74.95,2869.85,2017-06-05,38,Sim,Junho,Atrasado
3,Gael Teixeira,Feminino,Nao,Fibra,Nao,Sim,Mensal,CartaoCredito,Sim,Nao,Sim,Nao,0,Nao,Sim,Nao,Sim,55.90,238.50,2021-01-25,4,Nao,Janeiro,Atrasado
4,Manuela Teixeira,Feminino,Sim,Fibra,Nao,Nao,Mensal,DebitoAutomatico,Nao,Nao,Nao,Nao,1,Nao,Sim,Sim,Sim,53.45,119.50,2018-10-21,2,Nao,Outubro,Atrasado
5,Isadora Macedo,Masculino,Nao,DSL,Nao,Nao,Mensal,BoletoEletronico,Nao,Nao,Nao,Sim,0,Nao,Sim,Nao,Sim,49.85,3370.20,2019-02-09,70,Nao,Fevereiro,Atrasado


# SALVANDO A TABELA FORMATADA

In [15]:
df.to_csv(r'CleanDataBase.csv', index=False)